Get only review text for sentiment analysis

In [ ]:
import pandas as pd
import re
import math

data = pd.read_csv("./kinton_reviews.csv")
train = pd.read_csv("./data/train.csv")


# get reviews from after the ice cream promotion ended (estimation)
data1 = data[0:283]

# retrieve only the text
reviews1 = data1[["review_rating","review_text"]].dropna()

reviews1.rename(columns={'review_rating': 'rating', 'review_text': 'review'}, inplace=True) 


Start building sentiment analysis model

In [ ]:
from textblob import TextBlob
from deep_translator import GoogleTranslator
import langid

# use textblob to get whethere a review is positive or negative
reviews1["polarity"] = None

def getPolarity(text):
    text = str(text)
    
    # detect language
    try:
        lang = langid.classify(text)[0]
    except Exception:
        lang = 'en'
    
    if lang != 'en':
        text = GoogleTranslator(source = 'auto', target = 'en').translate(text = text)
    
    return TextBlob(text).sentiment.polarity

reviews1['polarity'] = reviews1['review'].apply(getPolarity)
reviews1['sentiment_model'] = reviews1['polarity'].apply(lambda x: 'positive' if x > 0.15 else ('negative' if x < -0.15 else 'neutral'))
reviews1['sentiment_true'] = reviews1['rating'].apply(lambda x: 'positive' if x > 3 else ('negative' if x < 3 else 'neutral'))
reviews1['accuracy'] = reviews1['sentiment_model'] == reviews1['sentiment_true']
    
    

Calculate prediction accuracy

In [ ]:
trues = reviews1['accuracy'].sum()

all = len(reviews1)

(trues / all) * 100

reviews1.head()
    

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns

kinton =  plt.figure(figsize=(8,5))

sns.boxplot(x="rating", y="polarity", data=reviews1)
plt.title("Distribution of Sentiment Polarity by Star Rating Using Kinton Reviews")
plt.xlabel("True Star Rating")
plt.ylabel("Polarity")
plt.show()


In [ ]:
# find percentage of food reviews

good = len(reviews1[reviews1['rating'] >= 4])

(good / len(reviews1)) * 100
